In [1]:
import pandas as pd

Cargamos la data necesaria

In [2]:
df_items = pd.read_parquet("../Datasets/australian_items_limpio.parquet")
df_reviews = pd.read_parquet("../Datasets/reviews_analisis_sentiment.parquet")
df_games = pd.read_parquet("../Datasets/steam_games_limpio.parquet")

sleccionamos columnas necesaria del dataframe reviews

In [3]:
df_reviews = df_reviews[["reviews_item_id", "reviews_recommend", "year", "sentiment_analysis"]]

In [4]:
df_reviews

,reviews_item_id,reviews_recommend,year,sentiment_analysis
0,1250,True,2011,2
1,251610,True,2014,2
3,250320,True,2013,2
4,211420,True,2014,1
5,249130,True,2014,1
...,...,...,...,...
231273,332310,True,2015,1
231291,440,True,2014,0
231293,304930,True,2014,2
231419,265630,True,2015,2


Seleccionamos las columnas del dataset games

In [5]:
df_games = df_games[["title", "id"]]

In [6]:
df_games

,title,id
0,Lost Summoner Kitty,761140
0,Lost Summoner Kitty,761140
0,Lost Summoner Kitty,761140
0,Lost Summoner Kitty,761140
0,Lost Summoner Kitty,761140
...,...,...
32132,Russian Roads,610660
32132,Russian Roads,610660
32132,Russian Roads,610660
32133,EXIT 2 - Directions,658870


Combinamos los dos dataframes

In [7]:
df_user_recomend = pd.merge(df_games, df_reviews, left_on="id", right_on="reviews_item_id", how="inner")

In [8]:
df_user_recomend

,title,id,reviews_item_id,reviews_recommend,year,sentiment_analysis
0,Half-Life,70,70,True,2015,1
1,Half-Life,70,70,True,2014,1
2,Half-Life,70,70,True,2015,1
3,Half-Life,70,70,True,2014,2
4,Half-Life,70,70,True,2011,2
...,...,...,...,...,...,...
100224,Counter-Strike: Condition Zero,80,80,False,2014,1
100225,Counter-Strike: Condition Zero,80,80,True,2013,2
100226,Counter-Strike: Condition Zero,80,80,True,2011,2
100227,Counter-Strike: Condition Zero,80,80,True,2014,1


In [13]:
df_user_recomend["year"] = df_user_recomend["year"].astype(int)

Creamos la funcion

In [16]:
def userRecoomend(año: int):
    games_year = df_user_recomend[df_user_recomend["year"]== año]
    recommends = games_year[(games_year["reviews_recommend"]== True) & (games_year["sentiment_analysis"]== 2)]
    
    # Agrupamos por juego recomendados
    games = recommends["title"].value_counts().reset_index()
    games.columns = ["games", "recommend_count"]
    
    # Ordenamos y obtenemos el top3
    top_games = games.nlargest(3, "recommend_count")
    
    resultado = [{"Puesto {}: {}".format(i + 1, row['games']): row['recommend_count']} for i, row in top_games.iterrows()]
    
    return resultado

In [17]:
userRecoomend(2011)

[{'Puesto 1: Team Fortress 2': 98},
 {'Puesto 2: Terraria': 60},
 {'Puesto 3: Portal 2': 36}]

Comprobamos la funcon y decidimos cargar el datasat en la carpeta data, donde se encuentra la data que consumen las funciones

In [18]:
df_user_recomend.to_parquet("../Funciones/data/userRecommend.parquet")